# Building a database for crime reports

The goal of this project is to create a database named `crimes_db` with a table `boston_crimes` with appropriate datatypes for storing the data from the `boston.csv` file. The table will be created inside a schema named `crimes`.

We will also create `readonly` and `readwrite` groups with the appropriate privileges and create a user for each group.

The first two rows from the `boston.csv` dataset are shown here:

| incident_number | offense_code |     description    |    date    | day_of_the_week |     lat     |     long     |
|:---------------:|:------------:|:------------------:|:----------:|:---------------:|:-----------:|:------------:|
| 1               | 619          | LARCENY ALL OTHERS | 2018-09-02 | Sunday          | 42.35779134 | -71.13937053 |
| 2               | 1402         | VANDALISM          | 2018-08-21 | Tuesday         | 42.30682138 | -71.06030035 |

The first column represents the identifier of the crime. The second contains a numeric identifier code for the committed crime. The third represents a description of the crime. The next two rows contain the date on which the crime happened and the corresponding day of the week. Finally, the last two columns represent the location of the crime with a latitude and longitude coordinates.

## Database And Schema

We will now create a database called `crime_db` with a schema called `crimes`:

In [1]:
import psycopg2
conn = psycopg2.connect(dbname='dq', user='dq')
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE DATABASE crime_db;")
conn.autocommit = False
conn.close()
conn = psycopg2.connect(dbname='crime_db', user='dq')
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes;")
conn.commit()
conn.close()

We will now open the `boston.csv` file to view column names and the first row.

In [2]:
import csv
with open('boston.csv', 'r') as f:
    reader = csv.reader(f)
    boston = []
    for row in reader:
        boston.append(row)
    header = boston[0]
    first_row = boston[1]
    print(header)
    print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


## Column Datatypes

We will create a function that will create a set of unique values for a column. We can use this to determine whether we can use an enumerated datatype or compute the maximum length of any text like column to select the right `VARCHAR` value.

In [3]:
def get_col_value_set(csv_filename, col_index):
    with open(csv_filename, 'r') as f:
        reader = csv.reader(f)
        next(reader)
        list = set()
        for row in reader:
            value = row[col_index]
            list.add(value)
        return list

for i in range(len(header)):
    values = get_col_value_set('boston.csv', i)
    print(header[i], len(values))

incident_number 298329
offense_code 219
description 239
date 1177
day_of_the_week 7
lat 18177
long 18177


We will now find the length of the longest text in the description column:

In [4]:
des_values = get_col_value_set('boston.csv', 2)
max_length = 0
for value in des_values:
    if max_length < len(value):
        max_length = len(value)
print(max_length)

58


The max length for description in 50 characters. We will set VARCHAR to 100 just incase there may be longer descriptions in the future.

The day_of_the_week column can have an enumerated datatype because it only has 7 fixed values.

In [5]:
conn = psycopg2.connect(dbname='crime_db', user='dq')
cur = conn.cursor()
cur.execute("""CREATE TYPE days AS ENUM (
            'Monday', 
            'Tuesday', 
            'Wednesday', 
            'Thursday', 
            'Friday', 
            'Saturday', 
            'Sunday'
            );
""")
conn.commit()
conn.close()

We will have a look at the values in the incident_number columns:

In [6]:
incident_values = []
with open('boston.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        incident_values.append(row[0])
print(incident_values[20])
print(incident_values[-1])

21
298329


We can see that the incident_number auto increments for every entry. We also saw that the number of unique values were 298329 which corresponds with the last entry. 

The incident_number column can have a integer datatype because of the range of 1 to 2147483647.

We will now look for the longest offense_code value:

In [7]:
offense_values = get_col_value_set('boston.csv', 1)
max_length = 0
for value in offense_values:
    if max_length < len(value):
        max_length = len(value)
print(max_length)

4


The offense_code column has a maximum of 4 digits and 219 unique values. We can rule out a enumerated datatype because there could be more offense codes.

The offense_code column could have the datatype smallint as it only takes up 2 bytes and has a range of -32768 to +32767.

The date column will have the datatype date as it follows the recommended format.

We will find the maximum and minimum values for the longitude and latitude columns:

In [8]:
lat_values = []
long_values = []
with open('boston.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        long_values.append(row[-1])
        lat_values.append(row[-2])      
print('Latitude: ' + min(lat_values) + ' - ' + max(lat_values))
print('Longitude: ' + min(long_values) + ' - ' + max(long_values))

Latitude: 42.2324133 - 42.39504158
Longitude: -70.96367615 - -71.17867378


We can use a decimal datatype for the lat and long column.

| Column Name | Datatype |
|---|---|
| incident_number | integer |
| offense_code | smallint |
| description | varchar(100) |
| date | date |
| day_of_the_week | enumarated |
| lat | decimal |
| long | decimal |


## Creating And Loading Table

In [9]:
conn = psycopg2.connect(dbname='crime_db', user='dq')
cur = conn.cursor()
cur.execute("""CREATE TABLE crimes.boston_crimes (
            incident_number INTEGER PRIMARY KEY,
            offense_code SMALLINT,
            description VARCHAR(100),
            date DATE,
            day_of_the_week DAYS,
            lat decimal,
            long decimal
            );
""")
conn.commit()
conn.close()

We have created the table `boston_crimes` above with the appropriate datatypes. We will now load the boston.csv file into the table.

In [10]:
conn = psycopg2.connect(dbname='crime_db', user='dq')
cur = conn.cursor()
with open('boston.csv') as f:
    cur.copy_expert("""COPY crimes.boston_crimes 
                    FROM STDIN WITH CSV HEADER;""", f)   
conn.commit()
conn.close()

In [11]:
conn = psycopg2.connect(dbname='crime_db', user='dq')
cur = conn.cursor()
cur.execute("""SELECT * FROM crimes.boston_crimes;""")
print(cur.fetchone())
conn.close()

(1, 619, 'LARCENY ALL OTHERS', datetime.date(2018, 9, 2), 'Sunday', Decimal('42.35779134'), Decimal('-71.13937053'))


## Revoking Public Privileges

In [12]:
conn = psycopg2.connect(dbname='crime_db', user='dq')
cur = conn.cursor()
cur.execute("""REVOKE ALL ON SCHEMA public FROM public;""")
cur.execute("""REVOKE ALL ON DATABASE crime_db FROM public;""")
conn.commit()
conn.close()

## Readonly Group

We will create a group called readonly and grant connection, usage and privileges.

In [38]:
conn = psycopg2.connect(dbname='crime_db', user='dq')
cur = conn.cursor()
cur.execute("""CREATE GROUP readonly NOLOGIN;""")
cur.execute("""GRANT CONNECT ON DATABASE crime_db TO readonly;""")
cur.execute("""GRANT USAGE ON SCHEMA crimes TO readonly;""")
cur.execute("""GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;""")
conn.commit()
conn.close()

## Readwrite Group

In [40]:
conn = psycopg2.connect(dbname='crime_db', user='dq')
cur = conn.cursor()
cur.execute("""CREATE GROUP readwrite NOLOGIN;""")
cur.execute("""GRANT CONNECT ON DATABASE crime_db TO readwrite;""")
cur.execute("""GRANT USAGE ON SCHEMA crimes TO readwrite;""")
cur.execute("""GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;""")
conn.commit()
conn.close()

## Users

We will now create a user called `data_analyst` for the `readonly` group and a user called `data_scientist` for the `readwrite` group.

In [26]:
import psycopg2
conn = psycopg2.connect(dbname='crime_db', user='dq')
cur = conn.cursor()
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readonly TO data_analyst;")
cur.execute("GRANT readwrite TO data_scientist;")
conn.commit()
conn.close()

## Testing

We will see what privileges have been granted our groups by looking at the `information_schema.table_privileges` table.

In [44]:
conn = psycopg2.connect(dbname='crime_db', user='dq')
cur = conn.cursor()
cur.execute("""
            SELECT grantee, privilege_type 
            FROM information_schema.table_privileges
            WHERE grantee IN ('readwrite', 'readonly')
            ;
        """)
privileges = cur.fetchall()
conn.close()
for row in privileges:
    print(row)

('readwrite', 'INSERT')
('readwrite', 'SELECT')
('readwrite', 'UPDATE')
('readwrite', 'DELETE')
('readonly', 'SELECT')


We will see the users have been created on our database by viewing the `pg_role` table.

- rolname: Role name
- rolsuper: Role has superuser privileges
- rolcreaterole: Role can create more roles
- rolcreatedb: Role can create databases
- rolcanlogin: Role can log in. That is, this role can be given as the initial session authorization identifier
- rolpassword: Not the password (always reads as ********)

In [52]:
conn = psycopg2.connect(dbname='crime_db', user='dq')
cur = conn.cursor()
cur.execute("""
            SELECT 
            rolname,
            rolsuper,
            rolcreaterole,
            rolcreatedb,
            rolcanlogin,
            rolpassword
            FROM pg_roles
            ;
        """)
users = cur.fetchall()
conn.close()
print(['rolname', 'rolsuper',
            'rolcreaterole',
            'rolcreatedb',
            'rolcanlogin',
            'rolpassword'])
for row in users:
    print(row)

['rolname', 'rolsuper', 'rolcreaterole', 'rolcreatedb', 'rolcanlogin', 'rolpassword']
('dq', True, True, True, True, '********')
('readonly', False, False, False, False, '********')
('readwrite', False, False, False, False, '********')
('data_analyst', False, False, False, True, '********')
('data_scientist', False, False, False, True, '********')


We will now look at groups and see what users they have assigned to them.

In [63]:
conn = psycopg2.connect(dbname='crime_db', user='dq')
cur = conn.cursor()
cur.execute("""
            SELECT
                pg_roles.rolname,
                pg_user.usename
            FROM pg_user 
            JOIN pg_auth_members ON (pg_user.usesysid=pg_auth_members.member) 
            JOIN pg_roles ON (pg_roles.oid=pg_auth_members.roleid) 
            WHERE pg_roles.rolname IN ('readonly', 'readwrite')
            ;
        """)
groups = cur.fetchall()
conn.close()
print(['GROUP', 'USER'])
for row in groups:
    print(row)

['GROUP', 'USER']
('readonly', 'data_analyst')
('readwrite', 'data_scientist')
